About the Dataset:

1.id: unique id for a news article
2.title: the title of a news article
3
author: author of the news article
4.text: the text of the article; could be incomplete
5.label: a label that marks whether the news article is real or fake:

 1: Fake news
 0: real News

Importing the Dependencies

In [1]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [2]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [3]:
# printing the stopwords in English
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

Data Pre-processing

In [8]:
# loading the dataset to a pandas DataFrame
news_dataset = pd.read_csv('/content/train.csv', encoding='latin-1', on_bad_lines='skip') # or encoding='ISO-8859-1'
# or
# news_dataset = pd.read_csv('/content/train.csv', encoding='latin-1', error_bad_lines=False) # or encoding='ISO-8859-1'

In [9]:
news_dataset.shape

(30230, 5)

In [10]:
# print the first 5 rows of the dataframe
news_dataset.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didnât Even See Comeyâs...,Darrell Lucus,House Dem Aide: We Didnât Even See Comeyâs...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [11]:
# counting the number of missing values in the dataset
news_dataset.isnull().sum()

,0
id,0
title,812
author,2863
text,76
label,29


In [16]:
# replacing the null values with empty string
news_dataset = news_dataset.fillna('')

In [18]:
# merging the author name and news title
news_dataset['content'] = news_dataset['author']+' '+news_dataset['title']

In [19]:
print(news_dataset['content'])

0        Darrell Lucus House Dem Aide: We Didnât Even...
1        Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo...
2        Consortiumnews.com Why the Truth Might Get You...
3        Jessica Purkiss 15 Civilians Killed In Single ...
4        Howard Portnoy Iranian woman jailed for fictio...
                               ...                        
30225    Jerome Hudson Rapper T.I.: Trump a âPoster C...
30226    Benjamin Hoffman N.F.L. Playoffs: Schedule, Ma...
30227    Michael J. de la Merced and Rachel Abrams Macy...
30228    Alex Ansary NATO, Russia To Hold Parallel Exer...
30229              David Swanson What Keeps the F-35 Alive
Name: content, Length: 30230, dtype: object


In [20]:
# separating the data & label
X = news_dataset.drop(columns='label', axis=1)
Y = news_dataset['label']

In [21]:
print(X)
print(Y)

          id                                              title  \
0          0  House Dem Aide: We Didnât Even See Comeyâs...   
1          1  FLYNN: Hillary Clinton, Big Woman on Campus - ...   
2          2                  Why the Truth Might Get You Fired   
3          3  15 Civilians Killed In Single US Airstrike Hav...   
4          4  Iranian woman jailed for fictional unpublished...   
...      ...                                                ...   
30225  20795  Rapper T.I.: Trump a âPoster Child For White...   
30226  20796  N.F.L. Playoffs: Schedule, Matchups and Odds -...   
30227  20797  Macyâs Is Said to Receive Takeover Approach ...   
30228  20798  NATO, Russia To Hold Parallel Exercises In Bal...   
30229  20799                          What Keeps the F-35 Alive   

                                          author  \
0                                  Darrell Lucus   
1                                Daniel J. Flynn   
2                             Consortiu

Stemming:

Stemming is the process of reducing a word to its Root word

In [25]:
port_stem = PorterStemmer()

In [26]:
def stemming(content):
    stemmed_content = re.sub('[^a-zA-Z]',' ',content)
    stemmed_content = stemmed_content.lower()
    stemmed_content = stemmed_content.split()
    stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
    stemmed_content = ' '.join(stemmed_content)
    return stemmed_content

In [27]:
news_dataset['content'] = news_dataset['content'].apply(stemming)

In [28]:
print(news_dataset['content'])

0        darrel lucu hous dem aid even see comey letter...
1        daniel j flynn flynn hillari clinton big woman...
2                   consortiumnew com truth might get fire
3        jessica purkiss civilian kill singl us airstri...
4        howard portnoy iranian woman jail fiction unpu...
                               ...                        
30225    jerom hudson rapper trump poster child white s...
30226    benjamin hoffman n f l playoff schedul matchup...
30227    michael j de la merc rachel abram maci said re...
30228    alex ansari nato russia hold parallel exercis ...
30229                            david swanson keep f aliv
Name: content, Length: 30230, dtype: object


In [29]:
#separating the data and label
X = news_dataset['content'].values
Y = news_dataset['label'].values

In [30]:
print(X)

['darrel lucu hous dem aid even see comey letter jason chaffetz tweet'
 'daniel j flynn flynn hillari clinton big woman campu breitbart'
 'consortiumnew com truth might get fire' ...
 'michael j de la merc rachel abram maci said receiv takeov approach hudson bay new york time'
 'alex ansari nato russia hold parallel exercis balkan'
 'david swanson keep f aliv']


In [31]:
print(Y)

['1' '0' '1' ... '0' '1' '1']


In [32]:
# converting the textual data to numerical data
vectorizer = TfidfVectorizer()
vectorizer.fit(X)

X = vectorizer.transform(X)

In [33]:
print(X)

  (0, 267)	0.2683216496004606
  (0, 2486)	0.36725857882974516
  (0, 2963)	0.24646923370354198
  (0, 3606)	0.36160716109305074
  (0, 3799)	0.2701102015737163
  (0, 4983)	0.23139118426619124
  (0, 7020)	0.2163567422582884
  (0, 7711)	0.24861261650124197
  (0, 8649)	0.29349129870543583
  (0, 8928)	0.36725857882974516
  (0, 13502)	0.25408121423487334
  (0, 15721)	0.28447260464590324
  (1, 1498)	0.2959557781167794
  (1, 1897)	0.15659771425359897
  (1, 2226)	0.38316362237635554
  (1, 2817)	0.1918046161526445
  (1, 3574)	0.2632845057281194
  (1, 5516)	0.7115505822045847
  (1, 6831)	0.1914441246305809
  (1, 16839)	0.30333174273730956
  (2, 2947)	0.3174585617621041
  (2, 3107)	0.46277218525555425
  (2, 5402)	0.38668415074835044
  (2, 5982)	0.3483272044712433
  (2, 9641)	0.4941432573914392
  :	:
  (30227, 3649)	0.21082751469616068
  (30227, 7058)	0.21750877247652928
  (30227, 8383)	0.22245112782306817
  (30227, 9007)	0.3625408758893759
  (30227, 9538)	0.2954141459709244
  (30227, 9609)	0.1756459

Splitting the dataset to training & test data

In [35]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, random_state=2)

Training the Model: Logistic Regression

In [37]:
model = LogisticRegression()

In [38]:
model.fit(X_train, Y_train)

LogisticRegression()

Evaluation

In [39]:
# accuracy score on the training data
X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(X_train_prediction, Y_train)

In [40]:
print('Accuracy score of the training data : ', training_data_accuracy)

Accuracy score of the training data :  0.9930119086999669


In [41]:
# accuracy score on the test data
X_test_prediction = model.predict(X_test)
test_data_accuracy = accuracy_score(X_test_prediction, Y_test)

In [42]:
print('Accuracy score of the test data : ', test_data_accuracy)

Accuracy score of the test data :  0.9885874958650347


Making a Predictive System

In [43]:
X_new = X_test[3]

prediction = model.predict(X_new)
print(prediction)

if (prediction[0]==0):
  print('The news is Real')
else:
  print('The news is Fake')

['1']
The news is Fake
